In [ ]:

!pip install accelerate==0.26.0
!pip install datasets
!pip install torch
!pip install evaluate
import torch
import evaluate
import sklearn.metrics
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    servic

First read the file.

In [ ]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")
goldstandard=dataset["validation"]["label"]
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_twitter.txt", 'w', encoding='utf-8') as f:
  for i in range(len(goldstandard)):
    f.write(str(goldstandard[i]))
    f.write('\n')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

Tokenize the text input.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
config = AutoConfig.from_pretrained("roberta-base")
config.num_labels = 3  # Set the number of classes
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", config=config)
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)
tokenized_training = dataset["train"].map(tokenize_function, batched=True)
tokenized_testing = dataset["validation"].map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

Set up the pre-trained model.

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # create Trainer
training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ipynb/final_project/roberta_twitter_checkpoint.txt",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)
def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_training,
  eval_dataset=tokenized_testing,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics
)


Train the model.

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
trainer.train()
model.save_pretrained("/content/drive/MyDrive/ipynb/final_project/roberta_twitter")

Epoch,Training Loss,Validation Loss,F1
1,0.485300,0.312019,0.891541
2,0.291200,0.280292,0.904941
3,0.199800,0.365666,0.903685


The downstream work can start here to save time. Use the model to produce predictions

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/ipynb/final_project/roberta_twitter")
predictions = trainer.predict(tokenized_testing)
preds = np.argmax(predictions.predictions, axis=1)


Save the labels for the downstream work.

In [ ]:
pre=preds.tolist()
with open("/content/drive/MyDrive/ipynb/final_project/roberta_twitter.txt", 'w', encoding='utf-8') as f:
  for i in range(len(pre)):
    f.write(str(pre[i]))
    f.write('\n')


The downstream work can start here to save time. Find the total accuracy.

In [ ]:
with open("/content/drive/MyDrive/ipynb/final_project/roberta_twitter.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
label=[]
for i in range(len(result)-1):
  label.append(int(result[i]))
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_twitter.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
goldstandard=[]
for i in range(len(result)-1):
  goldstandard.append(int(result[i]))
correct=0
for i in range(len(label)):
  if label[i]==goldstandard[i]:
    correct+=1
accuracy = correct / len(label)
print("Accuracy:", accuracy)

Accuracy: 0.9049413735343383


Find the Precise.

In [ ]:
print("Macro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='macro'))
print("Micro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='micro'))

Macro Precision: 0.8714847140270664
Micro Precision: 0.9049413735343383


Find the Recall

In [ ]:
print("Macro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='macro'))
print("Micro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='micro'))

Macro Recall: 0.8855063263880697
Micro Recall: 0.9049413735343383


Find the F1

In [ ]:
print("Macro F1:", sklearn.metrics.f1_score(goldstandard, label, average='macro'))
print("Micro F1:", sklearn.metrics.f1_score(goldstandard, label, average='micro'))

Macro F1: 0.8782796394817124
Micro F1: 0.9049413735343383
